# Data Lake for U.S. Immigration Office
### Data Engineering Capstone Project

#### Project Summary
In this project we are creating data lake about U.S. Immigrants and their behaviours which are related to city, temperature, airport.

Also we aim to ask some questions such as below:

* Why immigrants do choose these cities?
* Do they like mostly hot or warmer cities?
* Demographic structure is reason to choose these cities?

Therefore U.S. Immigrant Office will be able to take better decisions in their policies.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession

### Step 1: Scope the Project and Gather Data

#### Scope 
Our primary target is to create fact and dimension tables for data lake from different data sources and find answers of questions above.

After gathering data from data sources, first we wil store them in parquet files under directory which is "data_source/"

#### Describe and Gather Data 
##### 1 - I94 Immigration Data
This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. This is where the data comes from. There's a sample file so you can take a look at the data in csv format before reading it all in. You do not have to use the entire dataset, just use what you need to accomplish the goal you set at the beginning of the project. [Link](https://travel.trade.gov/research/reports/i94/historical/2016.html)

##### Data Dictionary for I94 Immigration Data
* **CICID:** Unique ID
* **I94YR:** 4 digit year
* **I94MON:** Numeric month
* **I94CIT:** Country of citizenship
* **I94RES:** Country of residence
* **I94PORT:** Port addmitted through
* **ARRDATE:** is the Arrival Date in the USA. It is a SAS date numeric field that a permament format has not been applied.
* **I94MODE:** 1 = Air; 2 = Sea; 3 = Land; 9 = Not reported
* **I94ADDR:** State of arrival
* **DEPDATE:** is the Departure Date from the USA.
* **I94BIR:** Age of Respondent in Years
* **I94VISA:** Visa codes collapsed into three categories 1 = Business, 2 = Pleasure, 3 = Student
* **COUNT:** Used for summary statistics
* **DTADFILE:** Character Date Field - Date added to I-94 Files
* **VISAPOST:** Department of State where where Visa was issued
* **OCCUP:** Occupation that will be performed in U.S.
* **ENTDEPA:** Arrival Flag - admitted or paroled into the U.S.
* **ENTDEPD:** Departure Flag - Departed, lost I-94 or is deceased
* **ENTDEPU:** Update Flag - Either apprehended, overstayed, adjusted to perm residence
* **MATFLAG:** Match flag - Match of arrival and departure records
* **BIRYEAR:** 4 digit year of birth
* **DTADDTO:** Character Date Field - Date to which admitted to U.S. (allowed to stay until)
* **GENDER:** Non-immigrant sex
* **INSNUM:** INS number
* **AIRLINE:** Airline used to arrive in U.S.
* **ADMNUM:** Admission Number
* **FLTNO:** Flight number of Airline used to arrive in U.S.
* **VISATYPE:** Class of admission legally admitting the non-immigrant to temporarily stay in U.S.

##### 2- World Temperature Data
This dataset came from Kaggle. You can read more about it here. [Link](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)
##### Data Dictionary for World Temperature Data
* **dt**
* **AverageTemperature**
* **AverageTemperatureUncertainty**
* **City**
* **Country**
* **Latitude**
* **Longitude**

##### 3- U.S. City Demographic Data
This data comes from OpenSoft. You can read more about it here. [Link](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)
##### Data Dictionary for U.S. City Demographic Data
* **City**
* **State**
* **Median Age**
* **Male Population**
* **Female Population**
* **Total Population**
* **Number of Veterans**
* **Foreign-born**
* **Average Household Size**
* **State Code**
* **Race**
* **Count**


##### 4- Airport Code Table
This is a simple table of airport codes and corresponding cities. It comes from here. [Link](https://datahub.io/core/airport-codes#data)
##### Data Dictionary for Airport Code Table
* **ident**
* **type**
* **name**
* **elevation_ft**
* **continent**
* **iso_country**
* **iso_region**
* **municipality**
* **gps_code**
* **iata_code**
* **local_code**
* **coordinates**

In [2]:
# Read in the data here
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

In [3]:
df_immigration = spark.read.format("com.github.saurfang.sas.spark").load("../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat")
df_immigration.createOrReplaceTempView("immigration")
df_immigration = spark.sql("SELECT * FROM immigration")

In [4]:
df_immigration.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [5]:
df_immigration.write.mode('overwrite').parquet("data_source/immigration.parquet")

In [6]:
df_immigration_parquet = spark.read.parquet("data_source/immigration.parquet")

In [7]:
df_immigration.count()

3096313

In [8]:
df_immigration_parquet.count()

3096313

In [9]:
df_temp = spark.read.format('csv').option("delimiter", ",").option("header", "true").load('../../data2/GlobalLandTemperaturesByCity.csv')
df_temp.createOrReplaceTempView("temperature")
df_temp = spark.sql("SELECT * FROM temperature")

In [10]:
df_temp.show(5)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows



In [11]:
df_temp.write.mode('overwrite').parquet("data_source/temperature.parquet")

In [12]:
df_temp_parquet = spark.read.parquet("data_source/temperature.parquet")

In [13]:
df_temp.count()

8599212

In [14]:
df_temp_parquet.count()

8599212

In [15]:
df_airport_codes = spark.read.format('csv').option("delimiter", ",").option("header", "true").load('airport-codes_csv.csv')
df_airport_codes.createOrReplaceTempView("airport_codes")
df_airport_codes = spark.sql("SELECT * FROM airport_codes")

In [16]:
df_airport_codes.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

In [17]:
df_airport_codes.write.mode('overwrite').parquet("data_source/airport_codes.parquet")

In [18]:
df_airport_codes_parquet = spark.read.parquet("data_source/airport_codes.parquet")

In [19]:
df_airport_codes.count()

55075

In [20]:
df_airport_codes_parquet.count()

55075

In [21]:
df_demography = spark.read.format('csv').option("delimiter", ";").option("header", "true").load('us-cities-demographics.csv')
df_demography.createOrReplaceTempView("demography")
df_demography = spark.sql("SELECT City, State, `Median Age` AS Median_Age, `Male Population` AS Male_Population,\
                          `Total Population` AS Total_Population, `Number of Veterans` AS Number_of_Veterans,\
                          `Foreign-born` AS Foreign_Born, `Average Household Size` AS Average_Household_Size,\
                          `State Code` AS State_Code, Race, Count \
                          FROM demography")

In [22]:
df_demography.show(5)

+----------------+-------------+----------+---------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median_Age|Male_Population|Total_Population|Number_of_Veterans|Foreign_Born|Average_Household_Size|State_Code|                Race|Count|
+----------------+-------------+----------+---------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040|           84839|              4819|        8229|                  2.58|        AL|        

In [23]:
df_demography.write.mode('overwrite').parquet("data_source/demography.parquet")

In [24]:
df_demography_parquet = spark.read.parquet("data_source/demography.parquet")

In [25]:
df_demography.count()

2891

In [26]:
df_demography_parquet.count()

2891

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [27]:
# Performing cleaning tasks here

In [28]:
# Getting data from sas label data and storing as parquet file
df_addr_label = spark.read.format('csv').option("delimiter", ";").option("header", "true").load('address.csv')
df_addr_label.write.mode('overwrite').parquet("stage_source/stg_address.parquet")
df_addr_label = spark.read.parquet("stage_source/stg_address.parquet")
df_addr_label.show(5)

+-------+----------+
|addr_id| addr_desc|
+-------+----------+
|     AL|   ALABAMA|
|     AK|    ALASKA|
|     AZ|   ARIZONA|
|     AR|  ARKANSAS|
|     CA|CALIFORNIA|
+-------+----------+
only showing top 5 rows



In [29]:
df_airport_label = spark.read.format('csv').option("delimiter", ";").option("header", "true").load('airport.csv')
df_airport_label.write.mode('overwrite').parquet("stage_source/stg_airport.parquet")
df_airport_label = spark.read.parquet("stage_source/stg_airport.parquet")
df_airport_label.show(5)

+----------+--------------------+-------------+
|airport_id|        airport_city|airport_state|
+----------+--------------------+-------------+
|       ALC|               ALCAN|           AK|
|       ANC|           ANCHORAGE|           AK|
|       BAR|BAKER AAF - BAKER...|           AK|
|       DAC|       DALTONS CACHE|           AK|
|       PIZ|DEW STATION PT LA...|           AK|
+----------+--------------------+-------------+
only showing top 5 rows



In [30]:
df_country_label = spark.read.format('csv').option("delimiter", ";").option("header", "true").load('country.csv')
df_country_label.write.mode('overwrite').parquet("stage_source/stg_country.parquet")
df_country_label = spark.read.parquet("stage_source/stg_country.parquet")
df_country_label.show(5)

+----------+--------------------+
|country_id|        country_desc|
+----------+--------------------+
|       582|  MEXICO Air Sea,...|
|       236|         AFGHANISTAN|
|       101|             ALBANIA|
|       316|             ALGERIA|
|       102|             ANDORRA|
+----------+--------------------+
only showing top 5 rows



In [31]:
df_mode_label = spark.read.format('csv').option("delimiter", ";").option("header", "true").load('mode.csv')
df_mode_label.write.mode('overwrite').parquet("stage_source/stg_mode.parquet")
df_mode_label = spark.read.parquet("stage_source/stg_mode.parquet")
df_mode_label.show(5)

+-------+------------+
|mode_id|   mode_desc|
+-------+------------+
|      1|         Air|
|      2|         Sea|
|      3|        Land|
|      9|Not reported|
+-------+------------+



In [32]:
df_visa_label = spark.read.format('csv').option("delimiter", ";").option("header", "true").load('visa.csv')
df_visa_label.write.mode('overwrite').parquet("stage_source/stg_visa.parquet")
df_visa_label = spark.read.parquet("stage_source/stg_visa.parquet")
df_visa_label.show(5)

+-------+---------+
|visa_id|visa_desc|
+-------+---------+
|      1| Business|
|      2| Pleasure|
|      3|  Student|
+-------+---------+



In [33]:
#Cleaning Temperature Data
df_temp = spark.read.parquet("data_source/temperature.parquet")
df_temp.createOrReplaceTempView("temperature")
df_temp = spark.sql("SELECT dt, AverageTemperature, AverageTemperatureUncertainty, \
                    UPPER(City) AS City, UPPER(Country) AS Country, \
                    Latitude,Longitude \
                    FROM temperature WHERE Country='United States' AND YEAR(dt)=2013 AND (AverageTemperature IS NOT NULL OR AverageTemperatureUncertainty IS NOT NULL)")
df_temp = df_temp.dropDuplicates(['dt', 'City'])

In [34]:
df_temp.show(5)

+----------+--------------------+-----------------------------+------------+-------------+--------+---------+
|        dt|  AverageTemperature|AverageTemperatureUncertainty|        City|      Country|Latitude|Longitude|
+----------+--------------------+-----------------------------+------------+-------------+--------+---------+
|2013-02-01|0.053999999999999986|                        0.198|INDEPENDENCE|UNITED STATES|  39.38N|   93.64W|
|2013-03-01|              12.543|                        0.225|   ARLINGTON|UNITED STATES|  32.95N|   96.70W|
|2013-03-01|  13.505999999999998|                        0.325| SANTA CLARA|UNITED STATES|  37.78N|  122.03W|
|2013-04-01|  6.8359999999999985|                         0.39|     LANSING|UNITED STATES|  42.59N|   85.09W|
|2013-05-01|              14.309|                        0.331|  MANCHESTER|UNITED STATES|  42.59N|   72.00W|
+----------+--------------------+-----------------------------+------------+-------------+--------+---------+
only showi

In [35]:
df_temp_pandas = df_temp.toPandas()
df_temp_pandas.isnull().sum()

dt                               0
AverageTemperature               0
AverageTemperatureUncertainty    0
City                             0
Country                          0
Latitude                         0
Longitude                        0
dtype: int64

In [36]:
df_temp.write.mode('overwrite').parquet("stage_source/stg_temperature.parquet")

In [37]:
#Cleaning Airport Data
df_airport_codes = spark.read.parquet("data_source/airport_codes.parquet")
df_airport_codes.createOrReplaceTempView("airport_codes")
df_airport_codes = spark.sql("SELECT ident, type, UPPER(name) AS name, elevation_ft, continent, iso_country,\
                        iso_region, UPPER(municipality) AS municipality, gps_code, iata_code, local_code,\
                        coordinates \
                       FROM airport_codes WHERE iso_country='US' AND type IN ('large_airport','medium_airport','small_airport')")

In [38]:
df_airport_codes.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
| 00AA|small_airport|AERO B RANCH AIRPORT|        3435|       NA|         US|     US-KS|       LEOTI|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        LOWELL FIELD|         450|       NA|         US|     US-AK|ANCHOR POINT|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        EPPS AIRPARK|         820|       NA|         US|     US-AL|     HARVEST|    00AL|     null|      00AL|-86.7703018188476...|
| 00AS|small_airport|      FULTON AIRPORT|        1100|       NA|         US|     

In [39]:
df_airport_pandas = df_airport_codes.toPandas()
df_airport_pandas.isnull().sum()

ident               0
type                0
name                0
elevation_ft       63
continent           0
iso_country         0
iso_region          0
municipality       50
gps_code          399
iata_code       12717
local_code        199
coordinates         0
dtype: int64

In [40]:
df_airport_codes.write.mode('overwrite').parquet("stage_source/stg_airport_codes.parquet")

In [41]:
#Cleaning Demographic Data
df_demography = spark.read.parquet("data_source/demography.parquet")
df_demography.createOrReplaceTempView("demography")
df_demography = spark.sql("SELECT UPPER(City) AS City, UPPER(State) AS State, \
                        Median_Age, Male_Population, Total_Population, Number_of_Veterans, \
                        Foreign_Born, Average_Household_Size, State_Code, Race, Count \
                        FROM demography WHERE Race IS NOT NULL OR City IS NOT NULL OR State_Code IS NOT NULL OR State IS NOT NULL")
df_demography = df_demography.dropDuplicates(['City', 'State', 'State_Code', 'Race'])

In [42]:
df_demography.show(5)

+--------------------+----------+----------+---------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|                City|     State|Median_Age|Male_Population|Total_Population|Number_of_Veterans|Foreign_Born|Average_Household_Size|State_Code|                Race|Count|
+--------------------+----------+----------+---------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|        APPLE VALLEY|CALIFORNIA|      34.3|          32873|           72185|              5714|        5801|                  3.03|        CA|  Hispanic or Latino|25928|
|ATHENS-CLARKE COU...|   GEORGIA|      26.5|          57415|          122563|              3953|       12868|                  2.44|        GA|  Hispanic or Latino|13159|
|            BERKELEY|CALIFORNIA|      32.5|          60142|          120971|              3736|       25000|                  2.35|        CA|Am

In [43]:
df_demography_pandas = df_demography.toPandas()

In [44]:
df_demography_pandas.isnull().sum()

City                       0
State                      0
Median_Age                 0
Male_Population            3
Total_Population           0
Number_of_Veterans        13
Foreign_Born              13
Average_Household_Size    16
State_Code                 0
Race                       0
Count                      0
dtype: int64

In [45]:
df_demography.write.mode('overwrite').parquet("stage_source/stg_demography.parquet")

In [46]:
#Cleaning Immigrant Data
df_immigrant = spark.read.parquet("data_source/immigration.parquet")
df_immigrant.createOrReplaceTempView("immigration")
df_addr_label.createOrReplaceTempView("addr_label")
df_airport_label.createOrReplaceTempView("airport_label")
df_country_label.createOrReplaceTempView("country_label")
df_mode_label.createOrReplaceTempView("mode_label")
df_visa_label.createOrReplaceTempView("visa_label")

df_immigrant = spark.sql("SELECT cicid AS Record_ID, i94yr AS Year, i94mon AS Month, \
                         CIT.country_desc AS Country_of_Citizen, RES.country_desc AS Country_of_Residence, \
                         AIR.airport_id AS Airport, date_add(to_date('1960-01-01'), arrdate) AS Arrival_Date, MOD.mode_desc AS Transfer_Mode,\
                         ADR.addr_desc AS Address, date_add(to_date('1960-01-01'), depdate) AS Departure_Date, i94bir AS Age, VIS.visa_desc AS Visa, \
                         biryear AS Birth_Year, gender AS Gender, airline AS Airline, visatype AS Visa_Type, \
                         UPPER(AIR.airport_city) AS City, UPPER(AIR.airport_state) AS State, count AS Number_of_Arrivals  \
                         FROM immigration F \
                         INNER JOIN country_label CIT ON F.i94cit=CIT.country_id \
                         INNER JOIN country_label RES ON F.i94res=RES.country_id \
                         INNER JOIN airport_label AIR ON F.i94port=AIR.airport_id \
                         INNER JOIN addr_label ADR ON F.i94addr=ADR.addr_id \
                         INNER JOIN mode_label MOD ON F.i94mode=MOD.mode_id \
                         INNER JOIN visa_label VIS ON F.i94visa=VIS.visa_id \
                         WHERE MOD.mode_desc='Air' AND F.gender IN ('F','M') \
                         AND F.i94bir IS NOT NULL AND F.airline IS NOT NULL \
                         AND UPPER(AIR.airport_state) IS NOT NULL")
                            
df_immigrant = df_immigrant.dropDuplicates(['Record_ID'])

In [47]:
df_immigrant.show(5)

+---------+------+-----+------------------+--------------------+-------+------------+-------------+-----------+--------------+----+--------+----------+------+-------+---------+----------------+-------+------------------+
|Record_ID|  Year|Month|Country_of_Citizen|Country_of_Residence|Airport|Arrival_Date|Transfer_Mode|    Address|Departure_Date| Age|    Visa|Birth_Year|Gender|Airline|Visa_Type|            City|  State|Number_of_Arrivals|
+---------+------+-----+------------------+--------------------+-------+------------+-------------+-----------+--------------+----+--------+----------+------+-------+---------+----------------+-------+------------------+
|    558.0|2016.0|  4.0|           AUSTRIA|             AUSTRIA|    SFR|  2016-04-01|          Air| CALIFORNIA|    2016-04-03|42.0|Business|    1974.0|     M|     LH|       WB|   SAN FRANCISCO|     CA|               1.0|
|    596.0|2016.0|  4.0|           AUSTRIA|             AUSTRIA|    NAS|  2016-04-01|          Air|    FLORIDA|    2

In [ ]:
df_immigrant_pandas = df_immigrant.toPandas()

In [ ]:
df_immigrant_pandas.isnull().sum()

Record_ID                   0
Year                        0
Month                       0
Country_of_Citizen          0
Country_of_Residence        0
Airport                     0
Arrival_Date                0
Transfer_Mode               0
Address                     0
Departure_Date          90647
Age                         0
Visa                        0
Birth_Year                  0
Gender                      0
Airline                     0
Visa_Type                   0
city                        0
state                       0
dtype: int64

In [48]:
df_immigrant.write.mode('overwrite').parquet("stage_source/stg_immigration.parquet")

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Star Schema is the selected model which is easy and effective to create queries by joining fact and dimension tables for analysing data.

##### Stage Tables

* stg_temperature
* stg_demoghraphy
* stg_airport
* stg_immigration

##### Dimension Tables

* dim_temperature
* dim_demography
* dim_airport
* dim_time

##### Fact Tables
* fct_immigration

#### 3.2 Mapping Out Data Pipelines
* Gathering data from several resources and stored under directory which is "data_source".
* Stage tables are created from data sources under directory which is "stage_source".
* null, duplicated values are removed from data sources.
* Aggregation is made if needed.
* Fact and dimension tables are created in parquet format under directory which is "fact_dimension" for data_lake.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [49]:
df_immigrant = spark.read.parquet("stage_source/stg_immigration.parquet")
df_temperature = spark.read.parquet("stage_source/stg_temperature.parquet")
df_airport = spark.read.parquet("stage_source/stg_airport_codes.parquet")
df_demography = spark.read.parquet("stage_source/stg_demography.parquet")

df_immigrant.createOrReplaceTempView("immigration")
df_temperature.createOrReplaceTempView("temperature")
df_airport.createOrReplaceTempView("airport")
df_demography.createOrReplaceTempView("demography")

In [50]:
# dim_time
df_time = spark.sql("""select distinct arrival_date as start_time, 
                                        hour(arrival_date) as hour, 
                                        day(arrival_date) as day, 
                                        weekofyear(arrival_date) as week, 
                                        month(arrival_date) as month, 
                                        year(arrival_date) as year, 
                                        dayofweek(arrival_date) as weekday 
                           from immigration
                       """)

In [51]:
df_time.show(5)

+----------+----+---+----+-----+----+-------+
|start_time|hour|day|week|month|year|weekday|
+----------+----+---+----+-----+----+-------+
|2016-04-29|   0| 29|  17|    4|2016|      6|
|2016-04-04|   0|  4|  14|    4|2016|      2|
|2016-04-05|   0|  5|  14|    4|2016|      3|
|2016-04-15|   0| 15|  15|    4|2016|      6|
|2016-04-27|   0| 27|  17|    4|2016|      4|
+----------+----+---+----+-----+----+-------+
only showing top 5 rows



In [52]:
df_time.printSchema()

root
 |-- start_time: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)



In [53]:
df_time.write.mode('overwrite').parquet("fact_dimension/dim_time.parquet")

In [54]:
# dim_temperature
df_temperature = spark.sql("SELECT City, AVG(AverageTemperature) AS Average_Temperature FROM temperature GROUP BY City")

In [55]:
df_temperature.show(5)

+------------+-------------------+
|        City|Average_Temperature|
+------------+-------------------+
|    STAMFORD| 12.163888888888888|
|    AMARILLO|  17.52955555555555|
|    SAVANNAH| 20.969111111111108|
|     ORLANDO|  23.77077777777778|
|INDIANAPOLIS| 13.398333333333333|
+------------+-------------------+
only showing top 5 rows



In [56]:
df_temperature.printSchema()

root
 |-- City: string (nullable = true)
 |-- Average_Temperature: double (nullable = true)



In [57]:
df_temperature.write.mode('overwrite').parquet("fact_dimension/dim_temperature.parquet")

In [58]:
# dim_airport
df_airport = spark.sql("SELECT * FROM airport")

In [59]:
df_airport.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
| 00AA|small_airport|AERO B RANCH AIRPORT|        3435|       NA|         US|     US-KS|       LEOTI|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        LOWELL FIELD|         450|       NA|         US|     US-AK|ANCHOR POINT|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        EPPS AIRPARK|         820|       NA|         US|     US-AL|     HARVEST|    00AL|     null|      00AL|-86.7703018188476...|
| 00AS|small_airport|      FULTON AIRPORT|        1100|       NA|         US|     

In [60]:
df_airport.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [61]:
df_airport.write.mode('overwrite').parquet("fact_dimension/dim_airport.parquet")

In [62]:
# dim_demography
df_demography = spark.sql("SELECT * FROM demography")

In [63]:
df_demography.show(5)

+----------------+--------------+----------+---------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|            City|         State|Median_Age|Male_Population|Total_Population|Number_of_Veterans|Foreign_Born|Average_Household_Size|State_Code|                Race| Count|
+----------------+--------------+----------+---------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|          AUBURN|    WASHINGTON|      37.1|          36837|           76580|              5401|       14842|                  2.73|        WA|Black or African-...|  4032|
|           BOISE|         IDAHO|      34.9|         110099|          218280|             16004|       13409|                  2.61|        ID|               White|204913|
|         BRANDON|       FLORIDA|      36.1|          55679|          113968|              9417|       16390|                  2.64|        

In [64]:
df_demography.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median_Age: string (nullable = true)
 |-- Male_Population: string (nullable = true)
 |-- Total_Population: string (nullable = true)
 |-- Number_of_Veterans: string (nullable = true)
 |-- Foreign_Born: string (nullable = true)
 |-- Average_Household_Size: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



In [70]:
df_demography.write.mode('overwrite').parquet("fact_dimension/dim_demography.parquet")

In [71]:
#dim_immigration
df_immigrant = spark.sql("SELECT * FROM immigration")

In [72]:
df_immigrant.show(5)

+---------+------+-----+------------------+--------------------+-------+------------+-------------+----------+--------------+----+--------+----------+------+-------+---------+----------------+-----+------------------+
|Record_ID|  Year|Month|Country_of_Citizen|Country_of_Residence|Airport|Arrival_Date|Transfer_Mode|   Address|Departure_Date| Age|    Visa|Birth_Year|Gender|Airline|Visa_Type|            City|State|Number_of_Arrivals|
+---------+------+-----+------------------+--------------------+-------+------------+-------------+----------+--------------+----+--------+----------+------+-------+---------+----------------+-----+------------------+
|    646.0|2016.0|  4.0|           AUSTRIA|             AUSTRIA|    CHI|  2016-04-01|          Air|  ILLINOIS|          null|37.0|Business|    1979.0|     M|     OS|       E2|         CHICAGO|   IL|               1.0|
|    898.0|2016.0|  4.0|           BELGIUM|             BELGIUM|    NEW|  2016-04-01|          Air|NEW JERSEY|    2016-04-08|13.

In [73]:
df_immigrant.printSchema()

root
 |-- Record_ID: double (nullable = true)
 |-- Year: double (nullable = true)
 |-- Month: double (nullable = true)
 |-- Country_of_Citizen: string (nullable = true)
 |-- Country_of_Residence: string (nullable = true)
 |-- Airport: string (nullable = true)
 |-- Arrival_Date: date (nullable = true)
 |-- Transfer_Mode: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Departure_Date: date (nullable = true)
 |-- Age: double (nullable = true)
 |-- Visa: string (nullable = true)
 |-- Birth_Year: double (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Visa_Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Number_of_Arrivals: double (nullable = true)



In [74]:
df_immigrant.write.mode('overwrite').parquet("fact_dimension/fct_immigration.parquet")

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [75]:
# Perform quality checks here
df_immigrant = spark.read.parquet("fact_dimension/fct_immigration.parquet")
df_temperature = spark.read.parquet("fact_dimension/dim_temperature.parquet")
df_airport = spark.read.parquet("fact_dimension/dim_airport.parquet")
df_demography = spark.read.parquet("fact_dimension/dim_demography.parquet")

In [76]:
if df_immigrant.count()>0:
    print("Data quality check passed. Immigrant table contains "+str(df_immigrant.count())+" rows.")
else:
    print("Data quality check failed. No data.")
    
if df_temperature.count()>0:
    print("Data quality check passed. Temperature table contains "+str(df_temperature.count())+" rows.")
else:
    print("Data quality check failed. No data.")
    
if df_airport.count()>0:
    print("Data quality check passed. Airport table contains "+str(df_airport.count())+" rows.")
else:
    print("Data quality check failed. No data.")
    
if df_demography.count()>0:
    print("Data quality check passed. Demography table contains "+str(df_demography.count())+" rows.")
else:
    print("Data quality check failed. No data.")

Data quality check passed. Immigrant table contains 2154775 rows.
Data quality check passed. Temperature table contains 248 rows.
Data quality check passed. Airport table contains 14582 rows.
Data quality check passed. Demography table contains 2891 rows.


#### 4.3 Data dictionary 
#### dim_time
 - **start_time:** date (date)
 - **hour:** integer (hour)
 - **day:** integer (day)
 - **week:** integer (week)
 - **month:** integer (month)
 - **year:** integer (year)
 - **weekday:** integer (weekday)

#### dim_temperature
- **City:** string (Name of city in USA)
- **Average_Temperature:** double (Average Temperature for each city)

#### dim_airport
- **ident:** string (Unique identifier)
- **type:** string (Type of the airport)
- **name:** string (Airport Name)
- **elevation_ft:** string (Altitude of the airport)
- **continent:** string (Continent)
- **iso_country:** string (ISO code of the country of the airport)
- **iso_region:** string (ISO code of the country of the airport)
- **municipality:** string (Airport City)
- **gps_code:** string (GPS code of the airport)
- **iata_code:** string (IATA code of the airport)
- **local_code:** string (Local code of the airport)
- **coordinates:** string (GPS coordinates of the airport)

#### dim_demography
- **City:** string (US City)
- **State:** string (US State)
- **Median_Age:** string (Median Age of Population)
- **Male_Population:** string (Number of Male Population)
- **Total_Population:** string (Number of Total Population)
- **Number_of_Veterans:** string (Number of Veterans)
- **Foreign_Born:** string (Number of People who borns not in USA)
- **Average_Household_Size:** string (Average Size of Houses)
- **State_Code:** string (US State Code)
- **Race:** string (Race Type)
- **Count:** string (Number of People)

#### fct_immigration
- Record_ID: double (Unique Identifier)
- Year: double (Year)
- Month: double (Month)
- Country_of_Citizen: string (Born Country)
- Country_of_Residence: string (Residence Country)
- Airport: string (Airport Code)
- Arrival_Date: date (Arrival Date in USA)
- Transfer_Mode: string (Transportation Type)
- Address: string (Residence Address)
- Departure_Date: date (Departure Date from USA)
- Age: double (Age)
- Visa: string (Visa Code)
- Birth_Year: double (Birth Year of Immigrant)
- Gender: string (Sex)
- Airline: string (Airways)
- Visa_Type: string (Visa Type of USA on Passport)
- City: string (Airport City)
- State: string (Airport State)

#### 4.4 Some Queries and Analysing Data
##### Let's query on data for our questions
First we will read our fact and dimension tables under directory which is "fact_dimension", in parquet format.

In [151]:
df_immigrant = spark.read.parquet("fact_dimension/fct_immigration.parquet")
df_temperature = spark.read.parquet("fact_dimension/dim_temperature.parquet")
df_airport = spark.read.parquet("fact_dimension/dim_airport.parquet")
df_demography = spark.read.parquet("fact_dimension/dim_demography.parquet")

df_immigrant.createOrReplaceTempView("immigration")
df_temperature.createOrReplaceTempView("temperature")
df_airport.createOrReplaceTempView("airport")
df_demography.createOrReplaceTempView("demography")

- **dim_temperature:** It consists of only U.S. cities. Here we wil use "City" column to join other datasets.
- **dim_demography:** It consists of only demographic structure of U.S. cities. "City" and "State" colums will be used for joining other datasets.
- **dim_airport:** It consists of only U.S. airports that are small, medium, large airports. Here "municipality" column will be joined over "City" columns with other datasets.
- **dim_time:** It is time dimension tables which is created by immigrant datasets. Here we used "arrival_date" to create time dimension table.

I will try join datasets with city or state columns. So let's see what happens.

First let's check city temperature - number of arrivals relationship.

I will accept 18 celcius degree as threshold. It means If city is higher than or equal 18, it is warm place else it is cold.
So I will assign a flag for this cities in query.

In [168]:
df_test_arrival_temp_relationship = spark.sql("SELECT B.City, B.Average_Temperature,\
                    B.Number_of_Arrivals, B.Total_Arrivals, (B.Number_of_Arrivals/B.Total_Arrivals)*100 AS Ratio, \
                    CASE WHEN B.Average_Temperature>=18 THEN 'Warm' ELSE 'Cold' END AS Temp_Flag \
                    FROM ( \
                    SELECT A.City, A.Average_Temperature, SUM(A.Number_of_Arrivals) AS Number_of_Arrivals, SUM(A.Total_Arrivals) AS Total_Arrivals \
                    FROM ( \
                    SELECT T.City, T.Average_Temperature, SUM(F.Number_of_Arrivals) AS Number_of_Arrivals, \
                    (SELECT SUM(K.Number_of_Arrivals) FROM immigration K INNER JOIN temperature R ON K.City=R.City) AS Total_Arrivals \
                    FROM immigration F INNER JOIN temperature T ON F.City=T.City \
                    GROUP BY T.City, T.Average_Temperature \
                    ) A GROUP BY A.City, A.Average_Temperature \
                    ORDER BY 3 DESC \
                    ) B ORDER BY 3 DESC")

In [180]:
df_test_arrival_temp_relationship.show(20)

+---------------+-------------------+------------------+--------------+------------------+---------+
|           City|Average_Temperature|Number_of_Arrivals|Total_Arrivals|             Ratio|Temp_Flag|
+---------------+-------------------+------------------+--------------+------------------+---------+
|       NEW YORK| 12.163888888888888|          347423.0|     1778460.0|19.535047175646344|     Cold|
|          MIAMI|  24.44011111111111|          269994.0|     1778460.0|15.181336662055935|     Warm|
|    LOS ANGELES| 18.120666666666665|          220832.0|     1778460.0|12.417034962833013|     Warm|
|  SAN FRANCISCO| 16.233666666666664|          116406.0|     1778460.0|6.5453257312506326|     Cold|
|        ORLANDO|  23.77077777777778|          115954.0|     1778460.0|  6.51991048435163|     Warm|
|        CHICAGO|  11.58688888888889|           92069.0|     1778460.0| 5.176894616690845|     Cold|
|        HOUSTON| 22.280222222222225|           81444.0|     1778460.0| 4.579467629297257| 

In [ ]:
df_test_arrival_temp_relationship.createOrReplaceTempView("arrival_temp")

In [177]:
df_percentage = spark.sql("SELECT Temp_Flag, SUM(Number_of_Arrivals), \
                          Total_Arrivals, \
                          (SUM(Number_of_Arrivals)/Total_Arrivals)*100 AS Ratio \
                          FROM arrival_temp \
                          GROUP BY Temp_Flag,Total_Arrivals")

In [178]:
df_percentage.show(5)

+---------+-----------------------+--------------+------------------+
|Temp_Flag|sum(Number_of_Arrivals)|Total_Arrivals|             Ratio|
+---------+-----------------------+--------------+------------------+
|     Cold|               838683.0|     1778460.0|47.157821935832125|
|     Warm|               939777.0|     1778460.0|52.842178064167875|
+---------+-----------------------+--------------+------------------+



According to results, ratios are nearly so close. So we can't say definitely immigrants choose cities because of climate.

#### Step 5: Complete Project Write Up
* As you see in the project, Spark is used for extracting, transforming and load operations since it has large data processing in memory compute. Also we can use Spark in cloud environment such as AWS EMR that have distributed nodes, processing data fast.

* If we had production environment, architecture would be like this below:

**AWS S3 --> AWS EMR (Spark Jobs) --> Apache Airflow --> AWS Redshift**

* According to data analysis of i94, monthly update is enough and we can schedule it via Apache Airflow in production environment.

 * **The data was increased by 100x:**
 We deploy our Spark solution to AWS EMR to compute more fast. Also cloud enviroment will scale itself if data becomes larger.
 
 * **The data populates a dashboard that must be updated on a daily basis by 7am every day:**
 We can use Apache Airflow to schedule and manage our data pipelines in production environment.
 
 * **The database needed to be accessed by 100+ people:**
 Our fact and dimension tables are in parquet format. So we can copy them to HDFS or AWS S3 and show them as external table by using Apache Hive. So people can connect Hive via their dashboard tools.